
PRACTICA REALIZADA POR:

* Manuel Cubertorer Gumbau
* Francisco Javier Corrales Estrella

En la asignatura de **Tipologia y ciclo de vida de los datos** de Universidad Oberta Catalunya Octubre 2021


Evaluación inicial

ROBOTS.txt: En primer lugar inspeccionamos el fichero robots.txt para ver la política sobre rastreos. El fichero no existe por lo que entendemos que el propietario del sitio web permite rastrear toda la página sin ningún tipo de restricción.

Mapa del sitio web (sitemap.xml). Tampoco se nos facilita un sitemap en el sitio web que nos facilite la navegación, por lo que la exploración ha sido manual.

Tamaño: En nuestro caso pretendemos recorrer partiendo de una pagina web los datos de un grid de tres pestañas distintas, y hacerlo navegando por los 30 dias anteriores.

Tecnología usada: Nuestra intención era usar las funcionalidades de la libreria BeautifulSoup, pero el contenido de la página está basado en Javascript (angularJs) y finalmente hemos tenido que usar Selenium




ROBOTS.txt: En primer lugar inspeccionamos el fichero robots.txt para ver la política sobre rastreos.

In [1]:
import builtwith
builtwith.parse('https://www.ree.es')

{'cms': ['Drupal'],
 'programming-languages': ['PHP'],
 'tag-managers': ['Google Tag Manager'],
 'javascript-graphics': ['Highcharts'],
 'widgets': ['Lockerz Share'],
 'web-frameworks': ['Twitter Bootstrap'],
 'javascript-frameworks': ['Underscore.js', 'jQuery', 'jQuery UI']}

En caso de no existir las librerias es posible que las tengamos que instalar

In [2]:
#pip install builtwith
#pip install selenium

Examinamos la tecnología con la que está construida la pagina web

In [3]:
import builtwith
builtwith.parse('https://demanda.ree.es/visiona/peninsula/demanda/tablas/2021-10-23/1')

{'javascript-frameworks': ['AngularJS', 'Modernizr', 'jQuery'],
 'javascript-graphics': ['D3', 'Highcharts'],
 'font-scripts': ['Google Font API'],
 'web-frameworks': ['Twitter Bootstrap']}

Inicialmente, vamos a intentar inspeccionar la página usando BeatifulSoup, pero como hemos podido comprobar en el apartado anterior, la página tiene contenido en javascript y lo mas probable es que el **contenido dinámico** no lo podamos rastrear con esta técnica.


Para poder descargar el contenido de las páginas vamos a definir una función que contenga 3 funcionalidades que podemos necesitar en un futuro ( en nuestro caso no aplica), son las siguientes:

* Setear un user-agent
* Política de reintentos
* Soporte de proxy

La idea de este código es del libro **Richard, Lawson. Web Scraping with Python, Packt Publishing, Limited, 2015.** El codigo viene para python 2 y lo hemos adaptado a python 3. también nos hemos apoyado para definir los reintentos en el siguiente enlace: https://www.peterbe.com/plog/best-practice-with-retries-with-requests


In [4]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

def requests_retry_session(user_agent='wswp', proxies=None, retries=3, backoff_factor=0.3, status_forcelist=(500, 502, 504),session=None,):
        
    session = requests.Session()
    if proxies:session.proxies.update(proxies)
    session.headers.update({'User-Agent': user_agent})

    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
        
    return session





In [5]:
from datetime import datetime, timedelta

#El formato de nuestra pagina es: https://demanda.ree.es/visiona/peninsula/demanda/tablas/2021-10-31/1
# Como nos interesa el dia anterior al actual la construimos dinámicamente

ayer_formateado = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')
print("El dia de ayer fue {}".format(ayer_formateado))

# Construimos las urls que queremos navegar
url_1 = "https://demanda.ree.es/visiona/peninsula/demanda/tablas/" + ayer_formateado + "/1"
url_2 = "https://demanda.ree.es/visiona/peninsula/demanda/tablas/" + ayer_formateado + "/2"
url_3 = "https://demanda.ree.es/visiona/peninsula/demanda/tablas/" + ayer_formateado + "/3"


page_1 = requests_retry_session().get(url_1)
page_2 = requests_retry_session().get(url_2)
page_3 = requests_retry_session().get(url_3)

# Lectura del html del response
from bs4 import BeautifulSoup
soup_1 = BeautifulSoup(page_1.content)
soup_2 = BeautifulSoup(page_2.content)
soup_3 = BeautifulSoup(page_3.content)

El dia de ayer fue 2021-11-03


Efectivamente, el contenido del html que se carga mediante BeatifulSoup no es lo mismo que vemos explorando la web con las devtools de Chrome. Pasamos a utilizar otra técnica como Selenium

In [6]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd

#opciones de navegacion
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

# CAMBIAR PATH DEL DRIVER SOLO AQUI
driver_path = '/home/fjcorrales/Master Ciencia de datos/tipologia y ciclo de vida de los datos/PRACTICA-1/lib/chromedriver_linux64/chromedriver'
#driver_path = '/users/Manuel/Downloads/chromedriver'

#Abrimos el navegador
driver = webdriver.Chrome(driver_path)

#Configuramos dias que vamos a recorrer
num_dias = 5

#Navegamos a la url
driver.get(url_1)

#Inicializamos los arrays para almacenar las variables
dias = list()
horas = list()
lectura_real = list()
lectura_prevista = list()
lectura_programada = list()
lectura_eolica = list()

#Navegamos por los 30 dias anteriores
for i in range(num_dias):            
    print("hoja",i)  
    # Leemos la información por horas    
    for t in range(20,164):
        # Escribimos este try para que en caso de no poder leer los datos de un día no se pare la lectura
        try:
            #Dia Hora
            xpath_diahora = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[1]"
            element_diahora = driver.find_element_by_xpath(xpath_diahora)
            dia = element_diahora.text.split()[0]
            hora = element_diahora.text.split()[1]
            dias.append(dia)
            horas.append(hora)

            #Real
            xpath_real = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[2]"
            element_real = driver.find_element_by_xpath(xpath_real)
            lectura_real.append(element_real.text)

            #Prevista
            xpath_prevista = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[3]"
            element_prevista = driver.find_element_by_xpath(xpath_prevista)
            lectura_prevista.append(element_prevista.text)

            #Programada
            xpath_programada = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[4]"
            element_programada = driver.find_element_by_xpath(xpath_programada)
            lectura_programada.append(element_programada.text)
        
        except:
            #Para evitar que los datos que no se pueden leer descuadren en el dataset se ponen vacios
            dias.append("")
            horas.append("")
            lectura_real.append("")
            lectura_prevista.append("")
            lectura_programada.append("")

    # Pulsamos click para navegar al dia anterior
    wait = WebDriverWait(driver, 5)
    elementButton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.md-icon-button main-menu md-button md-ink-ripple'.replace(' ','.'))))
    elementButton.click()


#Construimos el dataframe
df_1 = pd.DataFrame({'Fecha': dias, 'Hora': horas, 'Lectura Real': lectura_real, 'Lectura_prevista': lectura_prevista,\
                     'Lectura_programada': lectura_programada})

<ipython-input-6-7c0f421d100b>:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


hoja 0


<ipython-input-6-7c0f421d100b>:43: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_diahora = driver.find_element_by_xpath(xpath_diahora)
<ipython-input-6-7c0f421d100b>:51: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_real = driver.find_element_by_xpath(xpath_real)
<ipython-input-6-7c0f421d100b>:56: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_prevista = driver.find_element_by_xpath(xpath_prevista)
<ipython-input-6-7c0f421d100b>:61: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_programada = driver.find_element_by_xpath(xpath_programada)


hoja 1
hoja 2
hoja 3
hoja 4


In [7]:
#opciones de navegacion
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

#Abrimos el navegador
driver = webdriver.Chrome(driver_path)

#Navegamos a la url
driver.get(url_2)

#Inicializamos los arrays para almacenar las variables
lectura_eolica = list()
lectura_nuclear = list()
lectura_fuel_gas = list()
lectura_carbon = list()
lectura_ciclo_combinado = list()
lectura_hidraulica = list()
lectura_intercambios = list()
lectura_enlace_balear = list()
lectura_solar_fotovoltaica = list()
lectura_solar_termica = list()
lectura_termica_renovable = list()
lectura_cogeneracion_residuos = list()

#Navegamos por los 30 dias anteriores
for i in range(num_dias):            
    print("hoja",i)  
    # Leemos la información por horas    
    for t in range(20,164):
        # Escribimos este try para que en caso de no poder leer los datos de un día no se pare la lectura
        try:
            #Eólica
            xpath_eolica = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[2]"
            element_eolica = driver.find_element_by_xpath(xpath_eolica)
            lectura_eolica.append(element_eolica.text)

            #Nuclear
            xpath_nuclear = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[3]"
            element_nuclear = driver.find_element_by_xpath(xpath_nuclear)
            lectura_nuclear.append(element_nuclear.text)

            #Fuel/Gas
            xpath_fuel_gas = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[4]"
            element_fuel_gas = driver.find_element_by_xpath(xpath_fuel_gas)
            lectura_fuel_gas.append(element_fuel_gas.text)

            #Carbon
            xpath_carbon = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[5]"
            element_carbon = driver.find_element_by_xpath(xpath_carbon)
            lectura_carbon.append(element_carbon.text)

            #Ciclo combinado
            xpath_ciclo_combinado = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[6]"
            element_ciclo_combinado = driver.find_element_by_xpath(xpath_ciclo_combinado)
            lectura_ciclo_combinado.append(element_ciclo_combinado.text)

            #Hidraulica
            xpath_hidraulica = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[7]"
            element_hidraulica = driver.find_element_by_xpath(xpath_hidraulica)
            lectura_hidraulica.append(element_hidraulica.text)

            #Intercambios
            xpath_intercambios = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[8]"
            element_intercambios = driver.find_element_by_xpath(xpath_intercambios)
            lectura_intercambios.append(element_intercambios.text)

            #Enlace Balear
            xpath_enlace_balear = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[9]"
            element_enlace_balear = driver.find_element_by_xpath(xpath_enlace_balear)
            lectura_enlace_balear.append(element_enlace_balear.text)

            #Solar Fotovoltaica
            xpath_solar_fotovoltaica = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[10]"
            element_solar_fotovoltaica = driver.find_element_by_xpath(xpath_solar_fotovoltaica)
            lectura_solar_fotovoltaica.append(element_solar_fotovoltaica.text)

            #Solar Termica
            xpath_solar_termica = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[11]"
            element_solar_termica = driver.find_element_by_xpath(xpath_solar_termica)
            lectura_solar_termica.append(element_solar_termica.text)

            #Termica Renovable
            xpath_termica_renovable = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[12]"
            element_termica_renovable = driver.find_element_by_xpath(xpath_termica_renovable)
            lectura_termica_renovable.append(element_termica_renovable.text)

            #Cogeneracion Residuos
            xpath_cogeneracion_residuos = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[13]"
            element_cogeneracion_residuos = driver.find_element_by_xpath(xpath_cogeneracion_residuos)
            lectura_cogeneracion_residuos.append(element_cogeneracion_residuos.text)
        
        except:
            #Para evitar que los datos que no se pueden leer descuadren en el dataset se ponen vacios
            lectura_eolica.append("")
            lectura_nuclear.append("")
            lectura_fuel_gas.append("")
            lectura_carbon.append("")
            lectura_ciclo_combinado.append("")
            lectura_hidraulica.append("")
            lectura_intercambios.append("")
            lectura_enlace_balear.append("")
            lectura_solar_fotovoltaica.append("")
            lectura_solar_termica.append("")
            lectura_termica_renovable.append("")
            lectura_cogeneracion_residuos.append("")
            

    # Pulsamos click para navegar al dia anterior
    wait = WebDriverWait(driver, 5)
    elementButton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.md-icon-button main-menu md-button md-ink-ripple'.replace(' ','.'))))
    elementButton.click()


#Construimos el dataframe
df_2 = pd.DataFrame({'Eolica' : lectura_eolica, 'Nuclear' : lectura_nuclear, 'Fuel/Gas' : lectura_fuel_gas,\
                     'Carbon' : lectura_carbon, 'Ciclo Combinado' : lectura_ciclo_combinado, 'Hidraulica' : lectura_hidraulica,\
                     'Intercambios' : lectura_intercambios, 'Enlace Balear' : lectura_enlace_balear,\
                     'Solar Fotovoltaica' : lectura_solar_fotovoltaica, 'Solar Termica' : lectura_solar_termica,\
                     'Termica Renovable' : lectura_termica_renovable, 'Cogeneracion y Residuos' : lectura_cogeneracion_residuos})

<ipython-input-7-1c27f117d38b>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


hoja 0


<ipython-input-7-1c27f117d38b>:35: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_eolica = driver.find_element_by_xpath(xpath_eolica)


hoja 1


<ipython-input-7-1c27f117d38b>:40: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_nuclear = driver.find_element_by_xpath(xpath_nuclear)
<ipython-input-7-1c27f117d38b>:45: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_fuel_gas = driver.find_element_by_xpath(xpath_fuel_gas)
<ipython-input-7-1c27f117d38b>:50: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_carbon = driver.find_element_by_xpath(xpath_carbon)
<ipython-input-7-1c27f117d38b>:55: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_ciclo_combinado = driver.find_element_by_xpath(xpath_ciclo_combinado)
<ipython-input-7-1c27f117d38b>:60: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_hidraulica = driver.find_element_by_xpath(xpath_hidraulica

hoja 2
hoja 3
hoja 4


In [8]:
#opciones de navegacion
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

#Abrimos el navegador
driver = webdriver.Chrome(driver_path)

#Navegamos a la url
driver.get(url_3)

#Inicializamos los arrays para almacenar las variables
lectura_emisiones_carbon = list()
lectura_emisiones_ciclo_combinado = list()
lectura_emisiones_cogeneracion_residuos = list()

#Navegamos por los 30 dias anteriores
for i in range(num_dias):            
    print("hoja",i)  
    # Leemos la información por horas
    for t in range(20,164):
        # Escribimos este try para que en caso de no poder leer los datos de un día no se pare la lectura
        try:
            #Emisiones Carbon
            xpath_emisiones_carbon = "/html/body/div/md-content/div/section/div[2]/div[2]/table[3]/tbody/tr["+ str(t) +"]/td[5]"
            element_emisiones_carbon = driver.find_element_by_xpath(xpath_emisiones_carbon)
            lectura_emisiones_carbon.append(element_emisiones_carbon.text)

            #Emisiones Ciclo Combinado
            xpath_emisiones_ciclo_combinado = "/html/body/div/md-content/div/section/div[2]/div[2]/table[3]/tbody/tr["+ str(t) +"]/td[6]"
            element_emisiones_ciclo_combinado = driver.find_element_by_xpath(xpath_emisiones_ciclo_combinado)
            lectura_emisiones_ciclo_combinado.append(element_emisiones_ciclo_combinado.text)

            #Emisiones Cogeneracion y Residuos
            xpath_emisiones_cogeneracion_residuos = "/html/body/div/md-content/div/section/div[2]/div[2]/table[3]/tbody/tr["+ str(t) +"]/td[13]"
            element_emisiones_cogeneracion_residuos = driver.find_element_by_xpath(xpath_emisiones_cogeneracion_residuos)
            lectura_emisiones_cogeneracion_residuos.append(element_emisiones_cogeneracion_residuos.text)
        
        except:
            #Para evitar que los datos que no se pueden leer descuadren en el dataset se ponen vacios
            lectura_emisiones_carbon.append("")
            lectura_emisiones_ciclo_combinado.append("")
            lectura_emisiones_cogeneracion_residuos.append("")

    # Pulsamos click para navegar al dia anterior
    wait = WebDriverWait(driver, 5)
    elementButton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.md-icon-button main-menu md-button md-ink-ripple'.replace(' ','.'))))
    elementButton.click()


#Construimos el dataframe
df_3 = pd.DataFrame({'Emisiones Carbon': lectura_emisiones_carbon, 'Emisiones Ciclo Combinado': lectura_emisiones_ciclo_combinado,\
                     'Emisiones Cogeneracion y Residuos': lectura_emisiones_cogeneracion_residuos})

<ipython-input-8-a2cd22d33035>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


hoja 0


<ipython-input-8-a2cd22d33035>:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_emisiones_carbon = driver.find_element_by_xpath(xpath_emisiones_carbon)
<ipython-input-8-a2cd22d33035>:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_emisiones_ciclo_combinado = driver.find_element_by_xpath(xpath_emisiones_ciclo_combinado)
<ipython-input-8-a2cd22d33035>:36: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_emisiones_cogeneracion_residuos = driver.find_element_by_xpath(xpath_emisiones_cogeneracion_residuos)


hoja 1
hoja 2
hoja 3
hoja 4


Construimos el dataset final como la suma de los 3 dataframes que hemos generado.

In [9]:
dataset_final = pd.concat([df_1, df_2, df_3], axis=1)
dataset_final.to_csv('dataset_final.csv')
dataset_final.head(10)

,Fecha,Hora,Lectura Real,Lectura_prevista,Lectura_programada,Eolica,Nuclear,Fuel/Gas,Carbon,Ciclo Combinado,Hidraulica,Intercambios,Enlace Balear,Solar Fotovoltaica,Solar Termica,Termica Renovable,Cogeneracion y Residuos,Emisiones Carbon,Emisiones Ciclo Combinado,Emisiones Cogeneracion y Residuos
0,2021-11-03,00:00,23.962,24.424,23.846,,,,,,,,,,,,,649,2.009,898
1,2021-11-03,00:10,24.786,24.229,23.846,,,,,,,,,,,,,637,1.761,889
2,2021-11-03,00:20,24.465,24.035,23.846,,,,,,,,,,,,,632,1.664,886
3,2021-11-03,00:30,24.462,23.844,23.846,,,,,,,,,,,,,634,1.702,889
4,2021-11-03,00:40,24.038,23.655,23.846,,,,,,,,,,,,,642,1.703,882
5,2021-11-03,00:50,23.687,23.469,23.846,,,,,,,,,,,,,639,1.684,880
6,2021-11-03,01:00,23.518,23.284,22.850,,,,,,,,,,,,,640,1.683,880
7,2021-11-03,01:10,23.226,23.109,22.850,,,,,,,,,,,,,609,1.732,879
8,2021-11-03,01:20,23.141,22.947,22.850,,,,,,,,,,,,,614,1.674,877
9,2021-11-03,01:30,23.074,22.799,22.850,,,,,,,,,,,,,621,1.662,878


Exportamos los datos generados a un csv

In [15]:
#TODO: Subirlo a algún repositorio de datos
dataset_final.to_csv('../doc/file_name.csv')